In [ ]:
import pandas as pd

# Load dataset
data = pd.read_csv('./parkinsons_data-og.csv')

# Display first few rows and data types
data = data.drop(columns=['name'])
print(data.head())
print(data.info())


In [ ]:
# Check for missing values
print(data.isnull().sum())

# Example strategy to fill missing values (if any)
data.fillna(data.median(), inplace=True)


In [ ]:
# Discretize continuous columns (example using 4 bins for each feature)
for col in data.columns:
    if data[col].dtype != 'object':
        data[col] = pd.cut(data[col], bins=4, labels=False)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns (if any)
for col in data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])


In [ ]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=42)


In [ ]:
from pgmpy.estimators import HillClimbSearch, BicScore

# Using Hill Climb Search with BIC score
hc = HillClimbSearch(data)
model = hc.estimate(scoring_method=BicScore(data))
print(model.edges())


In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator

# Define the Bayesian Network model based on learned structure
bayesian_model = BayesianNetwork(model.edges())

# Learn CPTs
bayesian_model.fit(data, estimator=MaximumLikelihoodEstimator)


In [ ]:
from pgmpy.inference import VariableElimination

# Inference
inference = VariableElimination(bayesian_model)
query = inference.query(variables=['status'], evidence={'MDVP:Fo(Hz)': 1})  # Example query
print(query)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import (
    balanced_accuracy_score, accuracy_score, f1_score, roc_auc_score,
    brier_score_loss
)
from scipy.stats import entropy
from pgmpy.estimators import HillClimbSearch, BDeuScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination

# Load dataset
data = pd.read_csv('./parkinsons_data-og.csv')

data = data.drop(columns=['name'])

# Preprocess data: Handle missing values
data.fillna(data.median(), inplace=True)

# Discretize continuous variables (example with 4 bins for each feature)
n_bins = 4
for col in data.columns:
    if data[col].dtype != 'object':
        data[col] = pd.cut(data[col], bins=n_bins, labels=False)

# Encode categorical columns if any (convert to integer categories)
from sklearn.preprocessing import LabelEncoder

for col in data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

# Initialize k-fold cross-validation
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# To store results for all metrics
accuracies = []
balanced_accuracies = []
f1_scores = []
roc_aucs = []
brier_scores = []
kl_divergences = []
max_iter = 200000000000
# Cross-validation loop
for train_index, test_index in kf.split(data):
    train_data = data.iloc[train_index]
    test_data = data.iloc[test_index]

    # Structure learning using Hill Climb and BDeu score
    hc = HillClimbSearch(train_data)
    model = hc.estimate(scoring_method=BDeuScore(train_data), max_iter=max_iter)  # Use BDeu score
    
    # Define Bayesian Network with learned structure and fit CPTs
    bayesian_model = BayesianNetwork(model.edges())
    bayesian_model.fit(train_data, estimator=MaximumLikelihoodEstimator)
    
    # Inference setup
    inference = VariableElimination(bayesian_model)
    
    # Lists to store true and predicted values for metrics
    y_true = []
    y_pred = []
    y_prob = []

    # Testing the model
    for _, row in test_data.iterrows():
        # Example: Querying the 'status' variable
        evidence = row.drop('status').to_dict()  # Modify target variable if needed
        true_status = row['status']
        
        # Filter evidence to only include values within valid states for each variable
        valid_evidence = {}
        for var, value in evidence.items():
            if value < n_bins:  # Check if within expected range
                valid_evidence[var] = value
        
        # Perform inference to predict 'status'
        try:
            query_result = inference.map_query(variables=['status'], evidence=valid_evidence)
            # Store the true and predicted status
            y_true.append(true_status)
            y_pred.append(query_result['status'])

            try:
                # Perform the probability query
                prob_result = inference.query(variables=['status'], evidence=valid_evidence)
                prob_demented = prob_result.values[1]  # Probability of class 1
                y_prob.append(prob_demented)
            except KeyError as e:
                print(f"KeyError for evidence: {valid_evidence}. Missing key: {e}")
                y_prob.append(0.0000001)  # Append a very small value to avoid NaN
            
        except IndexError:
            # Skip this instance if evidence is out of range
            continue

    # Convert to numpy arrays for easier manipulation
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_prob = np.array(y_prob)

    # Check for NaN values and filter out any rows with NaNs
    valid_indices = ~np.isnan(y_true) & ~np.isnan(y_prob)
    y_true_clean = y_true[valid_indices]
    y_pred_clean = y_pred[valid_indices]
    y_prob_clean = y_prob[valid_indices]

    # Check the unique values in y_true_clean and y_pred_clean to determine pos_label
    unique_labels = set(y_true_clean) | set(y_pred_clean)
    pos_label = 1 if 1 in unique_labels else min(unique_labels)

    # Calculate metrics for this fold
    accuracies.append(accuracy_score(y_true_clean, y_pred_clean))
    balanced_accuracies.append(balanced_accuracy_score(y_true_clean, y_pred_clean))
    f1_scores.append(f1_score(y_true_clean, y_pred_clean, pos_label=pos_label))
    
    if len(y_prob_clean) > 0:  # Ensure there are valid probabilities
        roc_aucs.append(roc_auc_score(y_true_clean, y_prob_clean))
        brier_scores.append(brier_score_loss(y_true_clean, y_prob_clean))
        
        # Calculate KL Divergence for this fold
        # Add a small constant to avoid division by zero
        epsilon = 1e-10
        prob_true = np.array([p if y == pos_label else 1 - p for y, p in zip(y_true_clean, y_prob_clean)])
        prob_true = np.clip(prob_true, epsilon, None)  # Ensure no zero probabilities
        y_prob_clean = np.clip(y_prob_clean, epsilon, None)  # Ensure no zero probabilities

        kl_div = entropy(prob_true, y_prob_clean, base=2)
        kl_divergences.append(kl_div)

# Average the results across folds
print(f"Accuracy: {np.mean(accuracies):.4f}")
print(f"Balanced Accuracy: {np.mean(balanced_accuracies):.4f}")
print(f"F1 Score: {np.mean(f1_scores):.4f}")
print(f"ROC AUC: {np.mean(roc_aucs):.4f}")
print(f"Brier Score: {np.mean(brier_scores):.4f}")
print(f"KL Divergence: {np.mean(kl_divergences):.4f}")


In [ ]:
print(f"Accuracy: {np.mean(accuracies):.4f}")
print(f"Balanced Accuracy: {np.mean(balanced_accuracies):.4f}")
print(f"F1 Score: {np.mean(f1_scores):.4f}")
print(f"ROC AUC: {np.mean(roc_aucs):.4f}")
print(f"Brier Score: {np.mean(brier_scores):.4f}")
print(f"KL Divergence: {np.mean(kl_divergences):.4f}")

In [ ]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import (
    balanced_accuracy_score, accuracy_score, f1_score, roc_auc_score,
    brier_score_loss
)
from scipy.stats import entropy
from pgmpy.estimators import HillClimbSearch, BDeuScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination

# Load dataset
data = pd.read_csv('./parkinsons_data-og.csv')

data = data.drop(columns=['name'])

# Preprocess data: Handle missing values
data.fillna(data.median(), inplace=True)

# Discretize continuous variables (example with 4 bins for each feature)
n_bins = 4
for col in data.columns:
    if data[col].dtype != 'object':
        data[col] = pd.cut(data[col], bins=n_bins, labels=False)

# Encode categorical columns if any (convert to integer categories)
from sklearn.preprocessing import LabelEncoder

for col in data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

# Initialize k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# To store results for all metrics
accuracies = []
balanced_accuracies = []
f1_scores = []
roc_aucs = []
brier_scores = []
kl_divergences = []
max_iter = 200000000000
# Cross-validation loop
for train_index, test_index in kf.split(data):
    train_data = data.iloc[train_index]
    test_data = data.iloc[test_index]

    # Structure learning using Hill Climb and BDeu score
    hc = HillClimbSearch(train_data)
    model = hc.estimate(scoring_method=BDeuScore(train_data), max_iter=max_iter)  # Use BDeu score
    
    # Define Bayesian Network with learned structure and fit CPTs
    bayesian_model = BayesianNetwork(model.edges())
    bayesian_model.fit(train_data, estimator=MaximumLikelihoodEstimator)
    
    # Inference setup
    inference = VariableElimination(bayesian_model)
    
    # Lists to store true and predicted values for metrics
    y_true = []
    y_pred = []
    y_prob = []

    # Testing the model
    for _, row in test_data.iterrows():
        # Example: Querying the 'status' variable
        evidence = row.drop('status').to_dict()  # Modify target variable if needed
        true_status = row['status']
        
        # Filter evidence to only include values within valid states for each variable
        valid_evidence = {}
        for var, value in evidence.items():
            if value < n_bins:  # Check if within expected range
                valid_evidence[var] = value
        
        # Perform inference to predict 'status'
        try:
            query_result = inference.map_query(variables=['status'], evidence=valid_evidence)
            # Store the true and predicted status
            y_true.append(true_status)
            y_pred.append(query_result['status'])

            try:
                # Perform the probability query
                prob_result = inference.query(variables=['status'], evidence=valid_evidence)
                prob_demented = prob_result.values[1]  # Probability of class 1
                y_prob.append(prob_demented)
            except KeyError as e:
                print(f"KeyError for evidence: {valid_evidence}. Missing key: {e}")
                y_prob.append(0.0000001)  # Append a very small value to avoid NaN
            
        except IndexError:
            # Skip this instance if evidence is out of range
            continue

    # Convert to numpy arrays for easier manipulation
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_prob = np.array(y_prob)

    # Check for NaN values and filter out any rows with NaNs
    valid_indices = ~np.isnan(y_true) & ~np.isnan(y_prob)
    y_true_clean = y_true[valid_indices]
    y_pred_clean = y_pred[valid_indices]
    y_prob_clean = y_prob[valid_indices]

    # Check the unique values in y_true_clean and y_pred_clean to determine pos_label
    unique_labels = set(y_true_clean) | set(y_pred_clean)
    pos_label = 1 if 1 in unique_labels else min(unique_labels)

    # Calculate metrics for this fold
    accuracies.append(accuracy_score(y_true_clean, y_pred_clean))
    balanced_accuracies.append(balanced_accuracy_score(y_true_clean, y_pred_clean))
    f1_scores.append(f1_score(y_true_clean, y_pred_clean, pos_label=pos_label))
    
    if len(y_prob_clean) > 0:  # Ensure there are valid probabilities
        roc_aucs.append(roc_auc_score(y_true_clean, y_prob_clean))
        brier_scores.append(brier_score_loss(y_true_clean, y_prob_clean))
        
        # Calculate KL Divergence for this fold
        # Add a small constant to avoid division by zero
        epsilon = 1e-10
        prob_true = np.array([p if y == pos_label else 1 - p for y, p in zip(y_true_clean, y_prob_clean)])
        prob_true = np.clip(prob_true, epsilon, None)  # Ensure no zero probabilities
        y_prob_clean = np.clip(y_prob_clean, epsilon, None)  # Ensure no zero probabilities

        kl_div = entropy(prob_true, y_prob_clean, base=2)
        kl_divergences.append(kl_div)

# Average the results across folds
print(f"Accuracy: {np.mean(accuracies):.4f}")
print(f"Balanced Accuracy: {np.mean(balanced_accuracies):.4f}")
print(f"F1 Score: {np.mean(f1_scores):.4f}")
print(f"ROC AUC: {np.mean(roc_aucs):.4f}")
print(f"Brier Score: {np.mean(brier_scores):.4f}")
print(f"KL Divergence: {np.mean(kl_divergences):.4f}")


In [21]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import (
    balanced_accuracy_score, accuracy_score, f1_score, roc_auc_score,
    brier_score_loss
)
from scipy.stats import entropy
from pgmpy.estimators import HillClimbSearch, BDeuScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination

# Load dataset
data = pd.read_csv('./parkinsons_data-og.csv')

data = data.drop(columns=['name'])

# Preprocess data: Handle missing values
data.fillna(data.median(), inplace=True)

# Discretize continuous variables (example with 4 bins for each feature)
n_bins = 4
for col in data.columns:
    if data[col].dtype != 'object':
        data[col] = pd.cut(data[col], bins=n_bins, labels=False)

# Encode categorical columns if any (convert to integer categories)
from sklearn.preprocessing import LabelEncoder

for col in data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

# Initialize k-fold cross-validation
kf = KFold(n_splits=8, shuffle=True, random_state=51)

# To store results for all metrics
accuracies = []
balanced_accuracies = []
f1_scores = []
roc_aucs = []
brier_scores = []
kl_divergences = []
max_iter = 200000000000
# Cross-validation loop
for train_index, test_index in kf.split(data):
    train_data = data.iloc[train_index]
    test_data = data.iloc[test_index]

    # Structure learning using Hill Climb and BDeu score
    hc = HillClimbSearch(train_data)
    model = hc.estimate(scoring_method=BDeuScore(train_data), max_iter=max_iter)  # Use BDeu score
    
    # Define Bayesian Network with learned structure and fit CPTs
    bayesian_model = BayesianNetwork(model.edges())
    bayesian_model.fit(train_data, estimator=MaximumLikelihoodEstimator)
    
    # Inference setup
    inference = VariableElimination(bayesian_model)
    with open('./bayesian_model.pkl', 'wb') as f:
        pickle.dump(bayesian_model, f)
    # Lists to store true and predicted values for metrics
    y_true = []
    y_pred = []
    y_prob = []

    # Testing the model
    for _, row in test_data.iterrows():
        # Example: Querying the 'status' variable
        evidence = row.drop('status').to_dict()  # Modify target variable if needed
        true_status = row['status']
        
        # Filter evidence to only include values within valid states for each variable
        valid_evidence = {}
        for var, value in evidence.items():
            if value < n_bins:  # Check if within expected range
                valid_evidence[var] = value
        
        # Perform inference to predict 'status'
        try:
            query_result = inference.map_query(variables=['status'], evidence=valid_evidence,show_progress=False)
            # Store the true and predicted status
            y_true.append(true_status)
            y_pred.append(query_result['status'])

            # try:
                # Perform the probability query
            # print("true_status",true_status)
            # print("valid_evidence",valid_evidence)
            prob_result = inference.query(variables=['status'], evidence=valid_evidence, show_progress=False)
            prob_demented = prob_result.values[1]  # Probability of class 1
            y_prob.append(prob_demented)
            # except KeyError as e:
            #     print(f"KeyError for evidence: {valid_evidence}. Missing key: {e}")
            #     y_prob.append(0.0000001)  # Append a very small value to avoid NaN
            
        except IndexError:
            # Skip this instance if evidence is out of range
            continue

    # Convert to numpy arrays for easier manipulation
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_prob = np.array(y_prob)
    

    # Check for NaN values and filter out any rows with NaNs
    valid_indices = ~np.isnan(y_true) & ~np.isnan(y_prob)
    y_true_clean = y_true[valid_indices]
    y_pred_clean = y_pred[valid_indices]
    y_prob_clean = y_prob[valid_indices]

    # Check the unique values in y_true_clean and y_pred_clean to determine pos_label
    unique_labels = set(y_true_clean) | set(y_pred_clean)
    pos_label = 1 if 1 in unique_labels else min(unique_labels)

    # Calculate metrics for this fold
    accuracies.append(accuracy_score(y_true_clean, y_pred_clean))
    balanced_accuracies.append(balanced_accuracy_score(y_true_clean, y_pred_clean))
    f1_scores.append(f1_score(y_true_clean, y_pred_clean, pos_label=pos_label))
    
    if len(y_prob_clean) > 0:  # Ensure there are valid probabilities
        roc_aucs.append(roc_auc_score(y_true_clean, y_prob_clean))
        brier_scores.append(brier_score_loss(y_true_clean, y_prob_clean))
        
        # Calculate KL Divergence for this fold
        # Add a small constant to avoid division by zero
        epsilon = 1e-10
        prob_true = np.array([p if y == pos_label else 1 - p for y, p in zip(y_true_clean, y_prob_clean)])
        prob_true = np.clip(prob_true, epsilon, None)  # Ensure no zero probabilities
        y_prob_clean = np.clip(y_prob_clean, epsilon, None)  # Ensure no zero probabilities

        kl_div = entropy(prob_true, y_prob_clean, base=2)
        kl_divergences.append(kl_div)

# Average the results across folds
print(f"Accuracy: {np.mean(accuracies):.4f}")
print(f"Balanced Accuracy: {np.mean(balanced_accuracies):.4f}")
print(f"F1 Score: {np.mean(f1_scores):.4f}")
print(f"ROC AUC: {np.mean(roc_aucs):.4f}")
print(f"Brier Score: {np.mean(brier_scores):.4f}")
print(f"KL Divergence: {np.mean(kl_divergences):.4f}")


  0%|          | 0/200000000000 [00:00<?, ?it/s]

c:\Users\rohit\anaconda3\envs\AML\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:489: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi.values.sum()


  0%|          | 0/200000000000 [00:00<?, ?it/s]

  0%|          | 0/200000000000 [00:00<?, ?it/s]

c:\Users\rohit\anaconda3\envs\AML\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:489: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi.values.sum()
c:\Users\rohit\anaconda3\envs\AML\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:489: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi.values.sum()


...

c:\Users\rohit\anaconda3\envs\AML\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:489: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi.values.sum()


  0%|          | 0/200000000000 [00:00<?, ?it/s]

  0%|          | 0/200000000000 [00:00<?, ?it/s]

  0%|          | 0/200000000000 [00:00<?, ?it/s]

Accuracy: 0.8474
Balanced Accuracy: 0.7551
F1 Score: 0.6412
ROC AUC: 0.8427
Brier Score: 0.1218
KL Divergence: 3.6904


In [ ]:
import pandas as pd
import pickle
import numpy as np
from pgmpy.inference import VariableElimination

# Load the trained Bayesian model
with open('bayesian_model.pkl', 'rb') as f:
    bayesian_model = pickle.load(f)

# Set up inference
inference = VariableElimination(bayesian_model)

# Define your evidence with continuous values
# for status =1 
status = 1
evidence = {
    'MDVP:Fo(Hz)': 162.568,
    'MDVP:Fhi(Hz)': 198.346,
    'MDVP:Flo(Hz)': 77.63,
    'MDVP:Jitter(%)': 0.00502,
    'MDVP:Jitter(Abs)': 0.00003,
    'MDVP:RAP': 0.0028,
    'MDVP:PPQ': 0.00253,
    'Jitter:DDP': 0.00841,
    'MDVP:Shimmer': 0.01791,
    'MDVP:Shimmer(dB)': 0.168,
    'Shimmer:APQ3': 0.00793,
    'Shimmer:APQ5': 0.01057,
    'MDVP:APQ': 0.01799,
    'Shimmer:DDA': 0.0238,
    'NHR': 0.0117,
    'HNR': 25.678
}

#for status = 0
status = 0
evidence = {
    'MDVP:Fo(Hz)': 197.076,
    'MDVP:Fhi(Hz)': 206.896,
    'MDVP:Flo(Hz)': 192.055,
    'MDVP:Jitter(%)': 0.00289,
    'MDVP:Jitter(Abs)': 0.00001,
    'MDVP:RAP': 0.00166,
    'MDVP:PPQ': 0.00168,
    'Jitter:DDP': 0.00498,
    'MDVP:Shimmer': 0.01098,
    'MDVP:Shimmer(dB)': 0.097,
    'Shimmer:APQ3': 0.00563,
    'Shimmer:APQ5': 0.0068,
    'MDVP:APQ': 0.00802,
    'Shimmer:DDA': 0.01689,
    'NHR': 0.00339,
    'HNR': 26.775
}


# Load your training dataset to get the binning information
data = pd.read_csv('./parkinsons_data-og.csv')
data = data.drop(columns=['name'])

# Define number of bins (should match training configuration)
n_bins = 4

# Discretize evidence to match trained model bins
for col, value in evidence.items():
    if col in data.columns:
        # Get bin edges for this column
        bins = pd.cut(data[col], bins=n_bins, labels=False, retbins=True)[1]
        
        # Determine which bin the evidence value falls into
        evidence[col] = np.digitize(value, bins) - 1  # "-1" to zero-index

# Perform the query for P(status=1 | evidence)
try:
    prob_result = inference.query(variables=['status'], evidence=evidence)
    prob_status_1 = prob_result.values[status]  # Probability of status=1
    print(f"P(status={status} | evidence) = {prob_status_1:.6f}")
except KeyError as e:
    print(f"KeyError: {e}. Please check if evidence is within the trained model's range.")
    
print(bayesian_model.get_cardinality())  # Print the cardinalities of all variables in the model

    
# After cross-validation, train on the entire dataset for final evaluation or inference
final_hc = HillClimbSearch(data)
final_model = final_hc.estimate(scoring_method=BDeuScore(data), max_iter=max_iter)
final_bayesian_model = BayesianNetwork(final_model.edges())
final_bayesian_model.fit(data, estimator=MaximumLikelihoodEstimator)

print(evidence)
# Perform inference using the final model
inference = VariableElimination(final_bayesian_model)
prob_result = inference.query(variables=['status'], evidence=evidence)
print(f"P(status=1 | evidence) = {prob_result.values[1]:.6f}")



IndexError: index 3 is out of bounds for axis 0 with size 2

: 

In [ ]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import (
    balanced_accuracy_score, accuracy_score, f1_score, roc_auc_score,
    brier_score_loss
)
from scipy.stats import entropy
from pgmpy.estimators import HillClimbSearch, BDeuScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder


# Factor class to store CPTs and perform operations
class Factor:
    def __init__(self, variables, cpt):
        self.variables = variables  # List of variables involved in the factor
        self.cpt = cpt  # Conditional probability table (CPT)

    def marginalize(self, variable):
        """Marginalize out the variable from this factor."""
        if variable not in self.variables:
            return self  # No need to marginalize if the variable is not in this factor
        
        if isinstance(self.variables, set):
            self.variables = list(self.variables)  # Convert to list if it's a set

        var_idx = self.variables.index(variable)  # Now this should work
        new_vars = self.variables[:var_idx] + self.variables[var_idx+1:]
        new_cpt = np.sum(self.cpt, axis=var_idx)
        return Factor(new_vars, new_cpt)


    def restrict(self, variable, value):
        """ Restrict the factor to a specific value of the variable """
        print('variable',variable)
        print('self.variables',self.variables)
        print("value", value)
        print("variable not in self.variables",variable not in self.variables)
    
        if variable not in self.variables:
            # print(f"Variable '{variable}' not found in factor variables: {self.variables}")
            return self  # No need to restrict if the variable is not in this factor
        
        var_idx = self.variables.index(variable)
        new_vars = self.variables[:var_idx] + self.variables[var_idx+1:]
        # print("variable",variable)
        print("new_vars",new_vars)
        
        print("var_idx",var_idx)
        # print("self.variables",self.variables)
        # print("value",value)
        value_idx = np.where(self.variables[var_idx] == value)[0]
        print('value_idx',value_idx)
        
        new_cpt = self.cpt.take(value_idx, axis=var_idx)
        print(new_cpt)
        t = t[0]
        return Factor(new_vars, new_cpt)


class VariableElimination:
    def __init__(self, factors):
        self.factors = factors

    def eliminate(self, variable):
        """Eliminate a variable by summing over all its possible values."""
        relevant_factors = [factor for factor in self.factors if variable in factor.variables]
        
        if not relevant_factors:  # Check if no relevant factors were found
            print(f"No relevant factors found for variable {variable}. Skipping elimination.")
            return
        
        combined_factor = self.combine_factors(relevant_factors)
        marginalized_factor = combined_factor.marginalize(variable)
        self.factors = [factor for factor in self.factors if factor not in relevant_factors]
        self.factors.append(marginalized_factor)


    def combine_factors(self, factors):
        """ Combine a list of factors by multiplying their CPTs """
        # print("factors",factors)
        if not factors:  # Check if factors is empty
            raise ValueError("No relevant factors found for combining.")
        
        common_vars = set(factors[0].variables)
        for factor in factors[1:]:
            common_vars.intersection_update(factor.variables)
        
        combined_cpt = None
        for factor in factors:
            if combined_cpt is None:
                combined_cpt = factor.cpt
            else:
                combined_cpt = np.multiply(combined_cpt, factor.cpt)
        
        return Factor(common_vars, combined_cpt)

    def query(self, query_variables, evidence={}):
        """ Perform a query given a set of query variables and evidence """
        for variable, value in evidence.items():
            for factor in self.factors:
                factor = factor.restrict(variable, value)
        
        print('factor',factor.variables)
        t = t[0]
        
        non_query_variables = [var for factor in self.factors for var in factor.variables if var not in query_variables]
        for var in non_query_variables:
            self.eliminate(var)
        
        joint_cpt = None
        for factor in self.factors:
            if joint_cpt is None:
                joint_cpt = factor.cpt
            else:
                joint_cpt = np.multiply(joint_cpt, factor.cpt)
        
        return joint_cpt / np.sum(joint_cpt)



def calculate_cpt(data, parent, child, n_bins=10):
    """
    Calculate the Conditional Probability Table (CPT) for a parent-child pair.
    
    :param data: DataFrame containing the dataset
    :param parent: The parent variable (column name)
    :param child: The child variable (column name)
    :param n_bins: Number of bins for discretizing continuous data
    :return: A CPT (numpy array) for the given parent-child pair
    """
    # Discretize continuous variables by binning them (if necessary)
    data[parent] = pd.cut(data[parent], bins=n_bins, labels=False, include_lowest=True)
    data[child] = pd.cut(data[child], bins=n_bins, labels=False, include_lowest=True)
    
    # Initialize a dictionary to store the joint frequencies
    joint_freqs = defaultdict(int)
    
    # Calculate joint frequency of (parent_value, child_value) pairs
    for _, row in data.iterrows():
        parent_val = row[parent]
        child_val = row[child]
        joint_freqs[(parent_val, child_val)] += 1
    
    # Get the total number of samples for normalization
    total_samples = len(data)
    
    # Initialize the CPT matrix
    cpt = np.zeros((n_bins, n_bins))  # n_bins for both parent and child
    
    # Fill the CPT with conditional probabilities P(child | parent)
    for parent_val in range(n_bins):
        parent_count = sum(1 for val in data[parent] if val == parent_val)
        if parent_count == 0:
            continue  # Skip if no samples for this parent value
        
        for child_val in range(n_bins):
            joint_count = joint_freqs[(parent_val, child_val)]
            cpt[parent_val, child_val] = joint_count / parent_count  # P(child_val | parent_val)
    
    return cpt



In [ ]:

# Load dataset
data = pd.read_csv('./parkinsons_data-og.csv')
data = data.drop(columns=['name'])

# Preprocess data: Handle missing values
data.fillna(data.median(), inplace=True)

# Discretize continuous variables (example with 4 bins for each feature)

n_bins = 4
for col in data.columns:
    if data[col].dtype != 'object':
        data[col] = pd.cut(data[col], bins=n_bins, labels=False)

# Encode categorical columns if any (convert to integer categories)
for col in data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    

# Initialize k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# To store results for all metrics
accuracies = []
balanced_accuracies = []
f1_scores = []
roc_aucs = []
brier_scores = []
kl_divergences = []
max_iter = 200000000000



# Cross-validation loop
for train_index, test_index in kf.split(data):
    train_data = data.iloc[train_index]
    test_data = data.iloc[test_index]

    # Structure learning using Hill Climb and BDeu score
    hc = HillClimbSearch(train_data)
    model = hc.estimate(scoring_method=BDeuScore(train_data), max_iter=max_iter)  # Use BDeu score
    
    # Example: Iterate over all edges in the model to calculate the CPT for each edge
    factors = []

    for edge in model.edges():
        parent, child = edge
        cpt = calculate_cpt(data, parent, child, n_bins)  # Assuming you want 10 bins for discretization
        factors.append(Factor([parent, child], cpt))
   
    # Set up Variable Elimination
    ve = VariableElimination(factors)
    # Lists to store true and predicted values for metrics
    y_true = []
    y_pred = []
    y_prob = []

    # Testing the model
    for _, row in test_data.iterrows():
        evidence = row.drop('status').to_dict()  # Modify target variable if needed
        true_status = row['status']
        # Filter evidence to only include variables that exist in the factors
        valid_evidence = {}
        for var, value in evidence.items():
            for factor in factors:
                if var in factor.variables:
                    valid_evidence[var] = value
                    break  # Only include evidence for variables present in the factor list
        # Perform inference to predict 'status'
        try:
            query_result = ve.query(query_variables=['status'], evidence=valid_evidence)
            print("query_result",query_result)
            t=t[0]
            y_true.append(true_status)
            y_pred.append(np.argmax(query_result))  # Assuming 2 classes, take the max probability

            prob_demented = query_result[1]  # Probability of class 1 (Demented)
            y_prob.append(prob_demented)
        
        except KeyError as e:
            print(f"KeyError for evidence: {valid_evidence}. Missing key: {e}")
            y_prob.append(0.0000001)  # Append a very small value to avoid NaN

            
    # Convert to numpy arrays for easier manipulation
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_prob = np.array(y_prob)

    # Check for NaN values and filter out any rows with NaNs
    valid_indices = ~np.isnan(y_true) & ~np.isnan(y_prob)
    y_true_clean = y_true[valid_indices]
    y_pred_clean = y_pred[valid_indices]
    y_prob_clean = y_prob[valid_indices]

    # Calculate metrics for this fold
    accuracies.append(accuracy_score(y_true_clean, y_pred_clean))
    balanced_accuracies.append(balanced_accuracy_score(y_true_clean, y_pred_clean))
    f1_scores.append(f1_score(y_true_clean, y_pred_clean, pos_label=0, average='weighted'))
    
    
    if len(y_prob_clean) > 0:  # Ensure there are valid probabilities
        roc_aucs.append(roc_auc_score(y_true_clean, y_prob_clean))
        brier_scores.append(brier_score_loss(y_true_clean, y_prob_clean))
        
        # Calculate KL Divergence for this fold
        epsilon = 1e-10
        prob_true = np.array([p if y == 1 else 1 - p for y, p in zip(y_true_clean, y_prob_clean)])
        prob_true = np.clip(prob_true, epsilon, None)  # Ensure no zero probabilities
        y_prob_clean = np.clip(y_prob_clean, epsilon, None)  # Ensure no zero probabilities

        kl_div = entropy(prob_true, y_prob_clean, base=2)
        kl_divergences.append(kl_div)

# Average the results across folds
print(f"Accuracy: {np.mean(accuracies):.4f}")
print(f"Balanced Accuracy: {np.mean(balanced_accuracies):.4f}")
print(f"F1 Score: {np.mean(f1_scores):.4f}")
print(f"ROC AUC: {np.mean(roc_aucs):.4f}")
print(f"Brier Score: {np.mean(brier_scores):.4f}")
print(f"KL Divergence: {np.mean(kl_divergences):.4f}")



In [ ]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import (
    balanced_accuracy_score, accuracy_score, f1_score, roc_auc_score,
    brier_score_loss
)
from scipy.stats import entropy
from pgmpy.estimators import HillClimbSearch, BDeuScore, BayesianEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination

# Load dataset
data = pd.read_csv('./parkinsons_data-og.csv')
data = data.drop(columns=['name'])

# Preprocess data: Handle missing values
data.fillna(data.median(), inplace=True)

# Discretize continuous variables (example with 4 bins for each feature)
n_bins = 4
for col in data.columns:
    if data[col].dtype != 'object':
        data[col] = pd.cut(data[col], bins=n_bins, labels=False)

# Encode categorical columns if any (convert to integer categories)
from sklearn.preprocessing import LabelEncoder
for col in data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

# Initialize k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# To store results for all metrics
accuracies = []
balanced_accuracies = []
f1_scores = []
roc_aucs = []
brier_scores = []
kl_divergences = []
max_iter = 200000000000

# Cross-validation loop
for train_index, test_index in kf.split(data):
    train_data = data.iloc[train_index]
    test_data = data.iloc[test_index]

    # Structure learning using Hill Climb and BDeu score
    hc = HillClimbSearch(train_data)
    model = hc.estimate(scoring_method=BDeuScore(train_data), max_iter=max_iter)  # Use BDeu score
    
    # Define Bayesian Network with learned structure and fit CPTs using BayesianEstimator with Laplace smoothing
    bayesian_model = BayesianNetwork(model.edges())
    bayesian_model.fit(train_data, estimator=BayesianEstimator, prior_type="dirichlet", pseudo_counts=1)

    # Inference setup
    inference = VariableElimination(bayesian_model)
    with open('./bayesian_model.pkl', 'wb') as f:
        pickle.dump(bayesian_model, f)
    
    # Lists to store true and predicted values for metrics
    y_true = []
    y_pred = []
    y_prob = []

    # Testing the model
    for _, row in test_data.iterrows():
        evidence = row.drop('status').to_dict()  # Modify target variable if needed
        true_status = row['status']
        
        # Filter evidence to only include values within valid states for each variable
        valid_evidence = {var: value for var, value in evidence.items() if value < n_bins}
        
        # Perform inference to predict 'status'
        try:
            query_result = inference.map_query(variables=['status'], evidence=valid_evidence)
            y_true.append(true_status)
            y_pred.append(query_result['status'])

            try:
                # Perform the probability query
                prob_result = inference.query(variables=['status'], evidence=valid_evidence)
                prob_demented = prob_result.values[1]  # Probability of class 1
                y_prob.append(prob_demented)
            except KeyError as e:
                print(f"KeyError for evidence: {valid_evidence}. Missing key: {e}")
                y_prob.append(0.0000001)  # Append a very small value to avoid NaN
            
        except IndexError:
            continue

    # Convert to numpy arrays for easier manipulation
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_prob = np.array(y_prob)

    # Filter valid rows without NaNs
    valid_indices = ~np.isnan(y_true) & ~np.isnan(y_prob)
    y_true_clean = y_true[valid_indices]
    y_pred_clean = y_pred[valid_indices]
    y_prob_clean = y_prob[valid_indices]

    # Calculate metrics for this fold
    unique_labels = set(y_true_clean) | set(y_pred_clean)
    pos_label = 1 if 1 in unique_labels else min(unique_labels)

    accuracies.append(accuracy_score(y_true_clean, y_pred_clean))
    balanced_accuracies.append(balanced_accuracy_score(y_true_clean, y_pred_clean))
    f1_scores.append(f1_score(y_true_clean, y_pred_clean, pos_label=pos_label))
    
    if len(y_prob_clean) > 0:
        roc_aucs.append(roc_auc_score(y_true_clean, y_prob_clean))
        brier_scores.append(brier_score_loss(y_true_clean, y_prob_clean))
        
        # KL Divergence for this fold
        epsilon = 1e-10
        prob_true = np.array([p if y == pos_label else 1 - p for y, p in zip(y_true_clean, y_prob_clean)])
        prob_true = np.clip(prob_true, epsilon, None)
        y_prob_clean = np.clip(y_prob_clean, epsilon, None)

        kl_div = entropy(prob_true, y_prob_clean, base=2)
        kl_divergences.append(kl_div)

# Average the results across folds
print(f"Accuracy: {np.mean(accuracies):.4f}")
print(f"Balanced Accuracy: {np.mean(balanced_accuracies):.4f}")
print(f"F1 Score: {np.mean(f1_scores):.4f}")
print(f"ROC AUC: {np.mean(roc_aucs):.4f}")
print(f"Brier Score: {np.mean(brier_scores):.4f}")
print(f"KL Divergence: {np.mean(kl_divergences):.4f}")
